# Explore and meta-analyze the screens from Veronica's team with those from CRISPRBrain.

 - **Project:** CARD ARDIS.
 - **Author(s):** Mike Nalls + Veronica Ryan and James Harwot.

---
### Quick Description:

0.   Set up the environment.
1.   Explore the data.
2.   Harmonize the data.
3.   Meta-analyze the screens (each C-Brain screen to each veronica screen).
4.   FDR adjustments and similar.
5.   Anything novel?

Note, this code not pretty because I wrote it during a conference that I was presenting at. ;-)



# Set it up.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import os
from google.colab import drive
from scipy.stats import zscore
drive.mount('/content/drive/')
os.chdir("/screens_meta/")

Mounted at /content/drive/


# Explore the data.

In [ ]:
i3N_df = pd.read_csv("./veronica/rra-i3N.sgrna_summary.txt", delim_whitespace=True)
STMN2_df = pd.read_csv("./veronica/rra-High-Low-STMN2.sgrna_summary.txt", delim_whitespace=True)
i3N_high_df = pd.read_csv("./veronica/rra-ipsc-i3N-high.sgrna_summary.txt", delim_whitespace=True)
ipsc_i3N_low_df = pd.read_csv("./veronica/rra-ipsc-i3N-low.sgrna_summary.txt", delim_whitespace=True)
ipsc_df = pd.read_csv("./veronica/rra-ipsc.sgrna_summary.txt", delim_whitespace=True)

In [ ]:
cellrox_df = pd.read_csv("./cbrain/Glutamatergic Neuron-CellRox-CRISPRi.csv")
liperfluo_df = pd.read_csv("./cbrain/Glutamatergic Neuron-Liperfluo-CRISPRi.csv")

# Harmonize the data.

Here we try to make a standardized effect estimate from the phenotype score and LFCs. Then we derived estimated SEs.

In [ ]:
cellrox_df['beta'] = zscore(cellrox_df['Phenotype'])
liperfluo_df['beta'] = zscore(liperfluo_df['Phenotype'])
i3N_df['beta'] = zscore(i3N_df['LFC'])
STMN2_df['beta'] = zscore(STMN2_df['LFC'])
i3N_high_df['beta'] = zscore(i3N_high_df['LFC'])
ipsc_i3N_low_df['beta'] = zscore(ipsc_i3N_low_df['LFC'])
ipsc_df['beta'] = zscore(ipsc_df['LFC'])

Stabilize extreme p-values to floating points.

In [ ]:
cellrox_df['temp_p'] = cellrox_df['P Value']
liperfluo_df['temp_p'] = liperfluo_df['P Value']
i3N_df['temp_p'] = i3N_df['p.twosided']
STMN2_df['temp_p'] = STMN2_df['p.twosided']
i3N_high_df['temp_p'] = i3N_high_df['p.twosided']
ipsc_i3N_low_df['temp_p'] = ipsc_i3N_low_df['p.twosided']
ipsc_df['temp_p'] = ipsc_df['p.twosided']

In [ ]:
threshold = 1e-10

cellrox_df['temp_p'] = cellrox_df['temp_p'].apply(lambda x: max(x, threshold))
liperfluo_df['temp_p'] = liperfluo_df['temp_p'].apply(lambda x: max(x, threshold))
i3N_df['temp_p'] = i3N_df['temp_p'].apply(lambda x: max(x, threshold))
STMN2_df['temp_p'] = STMN2_df['temp_p'].apply(lambda x: max(x, threshold))
i3N_high_df['temp_p'] = i3N_high_df['temp_p'].apply(lambda x: max(x, threshold))
ipsc_i3N_low_df['temp_p'] = ipsc_i3N_low_df['temp_p'].apply(lambda x: max(x, threshold))
ipsc_df['temp_p'] = ipsc_df['temp_p'].apply(lambda x: max(x, threshold))

Note the Z scores calculated below are absolute value Z scores.

In [ ]:
cellrox_df['z_score'] = norm.ppf(1 - cellrox_df['temp_p'] / 2)
liperfluo_df['z_score'] = norm.ppf(1 - liperfluo_df['temp_p'] / 2)
i3N_df['z_score'] = norm.ppf(1 - i3N_df['temp_p'] / 2)
STMN2_df['z_score'] = norm.ppf(1 - STMN2_df['temp_p'] / 2)
i3N_high_df['z_score'] = norm.ppf(1 - i3N_high_df['temp_p'] / 2)
ipsc_i3N_low_df['z_score'] = norm.ppf(1 - ipsc_i3N_low_df['temp_p'] / 2)
ipsc_df['z_score'] = norm.ppf(1 - ipsc_df['temp_p'] / 2)

In [ ]:
cellrox_df['se'] = abs(cellrox_df['beta'] / cellrox_df['z_score'])
liperfluo_df['se'] = abs(liperfluo_df['beta'] / liperfluo_df['z_score'])
i3N_df['se'] = abs(i3N_df['beta'] / i3N_df['z_score'])
STMN2_df['se'] = abs(STMN2_df['beta'] / STMN2_df['z_score'])
i3N_high_df['se'] = abs(i3N_high_df['beta'] / i3N_high_df['z_score'])
ipsc_i3N_low_df['se'] = abs(ipsc_i3N_low_df['beta'] / ipsc_i3N_low_df['z_score'])
ipsc_df['se'] = abs(ipsc_df['beta'] / ipsc_df['z_score'])

In [ ]:
cellrox_df['p'] = cellrox_df['temp_p']
liperfluo_df['p'] = liperfluo_df['temp_p']
i3N_df['p'] = i3N_df['temp_p']
STMN2_df['p'] = STMN2_df['temp_p']
i3N_high_df['p'] = i3N_high_df['temp_p']
ipsc_i3N_low_df['p'] = ipsc_i3N_low_df['temp_p']
ipsc_df['p'] = ipsc_df['temp_p']

Quick reduce and some additional munging.

In [ ]:
reduced_cellrox_df = cellrox_df[['Gene','beta','se','z_score','p']]
reduced_liperfluo_df = liperfluo_df[['Gene','beta','se','z_score','p']]
reduced_i3N_df = i3N_df[['Gene','beta','se','z_score','p']]
reduced_STMN2_df = STMN2_df[['Gene','beta','se','z_score','p']]
reduced_i3N_high_df = i3N_high_df[['Gene','beta','se','z_score','p']]
reduced_ipsc_i3N_low_df = ipsc_i3N_low_df[['Gene','beta','se','z_score','p']]
reduced_ipsc_df = ipsc_df[['Gene','beta','se','z_score','p']]

Export these munged files.

In [ ]:
reduced_cellrox_df.to_csv("cellrox.csv")
reduced_liperfluo_df.to_csv("liperfluo.csv")
reduced_i3N_df.to_csv("i3N.csv")
reduced_STMN2_df.to_csv("STMN2.csv")
reduced_i3N_high_df.to_csv("i3N_hig.csv")
reduced_ipsc_i3N_low_df.to_csv("i3N_low.csv")
reduced_ipsc_df.to_csv("ipsc.csv")

# Meta_time.


First set some functions to speed this up.

In [ ]:
from statsmodels.stats.multitest import multipletests

def read_data(file_path):
    return pd.read_csv(file_path)

def merge_data(df1, df2):
    return pd.merge(df1, df2, on='Gene', suffixes=('_1', '_2'))

def fixed_effects(df):
    df['weight_1'] = 1 / df['se_1']**2
    df['weight_2'] = 1 / df['se_2']**2
    df['weighted_beta'] = (df['beta_1'] * df['weight_1'] + df['beta_2'] * df['weight_2']) / (df['weight_1'] + df['weight_2'])
    df['variance'] = 1 / (df['weight_1'] + df['weight_2'])
    df['fixed_effect_se'] = np.sqrt(df['variance'])
    df['fixed_effect_z'] = df['weighted_beta'] / df['fixed_effect_se']
    df['fixed_effect_p'] = norm.sf(abs(df['fixed_effect_z'])) * 2  # two-tailed test
    df['fixed_effect_p_FDR_adjusted'] = multipletests(df['fixed_effect_p'], method='fdr_bh')[1]
    return df


Fire off the metas for cellrox ...

In [ ]:
file_path_1 = 'cellrox.csv'
file_path_2 = 'i3N.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'cellrox.csv'
file_path_2 = 'STMN2.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'cellrox.csv'
file_path_2 = 'i3N_hig.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'cellrox.csv'
file_path_2 = 'i3N_low.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'cellrox.csv'
file_path_2 = 'ipsc.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

Fire off the metas for liperflou.

In [ ]:
file_path_1 = 'liperfluo.csv'
file_path_2 = 'i3N.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'liperfluo.csv'
file_path_2 = 'STMN2.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'liperfluo.csv'
file_path_2 = 'i3N_hig.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'liperfluo.csv'
file_path_2 = 'i3N_low.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

In [ ]:
file_path_1 = 'liperfluo.csv'
file_path_2 = 'ipsc.csv'
file_out = file_path_1.split('.', maxsplit=1)[0] + "_x_" + file_path_2.split('.', maxsplit=1)[0] + "_meta_analysis_results.csv"

# Process
df1 = read_data(file_path_1)
df2 = read_data(file_path_2)
merged_df = merge_data(df1, df2)

# Perform Fixed Effects Meta-Analysis
fe_results = fixed_effects(merged_df)

# Exports
fe_results.to_csv(file_out, index=False)

# Summarize
fe_results.describe()

# Now see if there are any novel hits.

In [ ]:
import pandas as pd
from statsmodels.stats.multitest import multipletests

# List of file paths
file_paths = ['liperfluo_x_STMN2_meta_analysis_results.csv', 'liperfluo_x_ipsc_meta_analysis_results.csv', 'liperfluo_x_i3N_meta_analysis_results.csv', 'liperfluo_x_i3N_low_meta_analysis_results.csv', 'liperfluo_x_i3N_hig_meta_analysis_results.csv', 'cellrox_x_STMN2_meta_analysis_results.csv', 'cellrox_x_ipsc_meta_analysis_results.csv', 'cellrox_x_i3N_meta_analysis_results.csv', 'cellrox_x_i3N_low_meta_analysis_results.csv', 'cellrox_x_i3N_hig_meta_analysis_results.csv']

# Loop through each file
for file_path in file_paths:
    # Read the data
    df = pd.read_csv(file_path)

    # Apply FDR adjustment for each specified column and add results to new columns
    for col in ['p_1', 'p_2']:
        adjusted_p_values = multipletests(df[col], method='fdr_bh')[1]
        df[f'{col}_FDR_adjusted'] = adjusted_p_values

    # Create the 'novel_hit' column based on specified conditions
    df['novel_hit'] = ((df['p_1_FDR_adjusted'] > 0.05) & (df['p_2_FDR_adjusted'] > 0.05) &
                       (df['fixed_effect_p_FDR_adjusted'] < 0.05)).astype(int)

    # Save the modified DataFrame to a binary format (pickle)
    out_path = "annotated_" + file_path
    df.to_csv(out_path, index=False)


    print(f"Processed {file_path} and saved to {out_path}.")
    print(df['novel_hit'].value_counts())


Processed liperfluo_x_STMN2_meta_analysis_results.csv and saved to annotated_liperfluo_x_STMN2_meta_analysis_results.csv.
0    22957
1      563
Name: novel_hit, dtype: int64
Processed liperfluo_x_ipsc_meta_analysis_results.csv and saved to annotated_liperfluo_x_ipsc_meta_analysis_results.csv.
0    23239
1      281
Name: novel_hit, dtype: int64
Processed liperfluo_x_i3N_meta_analysis_results.csv and saved to annotated_liperfluo_x_i3N_meta_analysis_results.csv.
0    23101
1      419
Name: novel_hit, dtype: int64
Processed liperfluo_x_i3N_low_meta_analysis_results.csv and saved to annotated_liperfluo_x_i3N_low_meta_analysis_results.csv.
0    23122
1      398
Name: novel_hit, dtype: int64
Processed liperfluo_x_i3N_hig_meta_analysis_results.csv and saved to annotated_liperfluo_x_i3N_hig_meta_analysis_results.csv.
0    23020
1      500
Name: novel_hit, dtype: int64
Processed cellrox_x_STMN2_meta_analysis_results.csv and saved to annotated_cellrox_x_STMN2_meta_analysis_results.csv.
0    22974